**torch.autograd**는 PyTorch에서 기울기 값을 구하는 시스템이다.

순전파 단계에서 autograd는:
* 텐서의 forward 연산이 될 때 실행된다.
* 연산 그래프를 동적으로 생성한다.
* 텐서 연산(ex: add, matmul) 을 수행하여 결과를 텐서로 돌려준다.
* 각 텐서의 `.grad_fn` 에 기울기 연산에 필요한 gradient function을 저장한다.

역전파 단계에서 autograd는:
* 결과 텐서 (Root)에서 .backward()가 호출될 때 시작한다.
* 순전파 때 만들어진 연산 그래프의 역방향을 따라가며 연쇄적으로 실행된다.
* `requires_grad=True`인 텐서에 대해서만 기울기를 계산한다.
* 각 텐서의 `.grad_fn` 으로 부터 기울기를 계산한다.
* 각 텐서의 `.grad` 속성에 계산된 기울기를 누적(accumulate)한다.
* 연쇄 법칙(chain rule)을 사용하여 모든 잎leaf 텐서들까지 전파propagate한다.

In [7]:
import torch

In [8]:
scalar_tensor = torch.tensor(5) # 스칼라 텐서
vector_tensor = torch.tensor([1,2,3,4]) # 벡터 텐서
matrix_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]]) # 행렬 텐서
tensor_3d = torch.tensor([[[1, 2, 3], [4, 5, 6]],
                          [[7, 8, 9], [10, 11, 12]]]) # 3D 텐서
tensor_ones = torch.ones(2, 3) # 2x3, 1로 초기화, 행렬 텐서
tensor_zeros = torch.zeros(2, 3) # 2x3, 0으로 초기화, 행렬 텐서
tensor_random = torch.randn(2, 3) # 2x3, 정규분포를 따르는 랜덤 값으로 초기화, 행렬 텐서
tensor_arange = torch.arange(0, 10) # [0~9]
tensor_linspace = torch.linspace(0, 1, steps=5) # [0.0, 0.25, 0.5, 0.75, 1.0]

In [9]:
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b # 1x5 dot 5x3 => 1x3 + 1x3 => 1x3
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

print("x:", x, type(x))
print("y:", y, type(y))
print("w:", w, type(w))
print("b:", b, type(b))
print("z:", z, type(z))
print("\ttorch.matmul:", torch.matmul, type(torch.matmul))
print("\tz.grad_fn:", z.grad_fn)
print("\tz.grad:", z.grad)
print("loss:", loss, type(loss))

x: tensor([1., 1., 1., 1., 1.]) <class 'torch.Tensor'>
y: tensor([0., 0., 0.]) <class 'torch.Tensor'>
w: tensor([[ 0.9698, -0.4488, -0.2511],
        [ 0.0960, -2.0260, -0.1918],
        [-0.6390, -0.2648,  1.2296],
        [ 0.9061, -0.0338, -1.8336],
        [ 0.5091,  0.0953, -0.0344]], requires_grad=True) <class 'torch.Tensor'>
b: tensor([-1.7136,  1.8476, -0.7694], requires_grad=True) <class 'torch.Tensor'>
z: tensor([ 0.1285, -0.8305, -1.8507], grad_fn=<AddBackward0>) <class 'torch.Tensor'>
	torch.matmul: <built-in method matmul of type object at 0x00007FFD01284D40> <class 'builtin_function_or_method'>
	z.grad_fn: <AddBackward0 object at 0x000001298561F580>
	z.grad: None
loss: tensor(0.4224, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) <class 'torch.Tensor'>


C:\Users\oiehot\AppData\Local\Temp\ipykernel_18752\401830688.py:15: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  print("\tz.grad:", z.grad)


In [10]:
"""
역전파 계산

성능상 이유로 backward는 한 번만 수행되는데 (수행 후 연산그래프가 삭제됨)
retain_graph를 True로 설정하면 여러번 수행할 수 있다.
"""
loss.backward(retain_graph=True)

print("w.grad:", w.grad)
print("b.grad:", b.grad)
print("x.requires_grad:", x.requires_grad) # False
print("y.requires_grad:", y.requires_grad) # False
print("w.requires_grad:", w.requires_grad) # True
print("b.requires_grad:", b.requires_grad) # True
print("z.requires_grad:", z.requires_grad) # True
print("loss.requires_grad:", loss.requires_grad) # True

w.grad: tensor([[0.1774, 0.1012, 0.0453],
        [0.1774, 0.1012, 0.0453],
        [0.1774, 0.1012, 0.0453],
        [0.1774, 0.1012, 0.0453],
        [0.1774, 0.1012, 0.0453]])
b.grad: tensor([0.1774, 0.1012, 0.0453])
x.requires_grad: False
y.requires_grad: False
w.requires_grad: True
b.requires_grad: True
z.requires_grad: True
loss.requires_grad: True


In [11]:
"""
컨텍스트 매니저를 사용하여
텐서를 기울기 연산에서 제외하기.
"""
with torch.no_grad():
	z = torch.matmul(x, w) + b
print(z.requires_grad)

False


In [12]:
"""
detach()를 사용하여 텐서를 기울기 연산에서 제외하기.
"""
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False
